In [1]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from ib_insync import *
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from optopus.ib_adapter import IBBrokerAdapter
from optopus.optopus import Optopus
from optopus.data_manager import DataManager, DataSource
from optopus.data_objects import IndexAsset, OptionChainAsset
from optopus.utils import pdo

%matplotlib inline

In [2]:
host = '127.0.0.1'
#port = 4002  # Gateway
port = 7497  # TWS
client = 75

util.startLoop()
ib = IB()
ib.connect(host, port, client)
opt = Optopus(IBBrokerAdapter(ib))

In [3]:
symbol = 'SPX'
contract = Index(symbol)
qcontract = ib.qualifyContracts(contract)[0]
qcontract

Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')

In [4]:
bars = ib.reqHistoricalData(
        qcontract,
        endDateTime='',
        durationStr='252 D',
        barSizeSetting='1 day',
        whatToShow='OPTION_IMPLIED_VOLATILITY',
        useRTH=True,
        formatDate=1)
df = util.df(bars)

In [5]:
df.tail(10)

,date,open,high,low,close,volume,barCount,average
242,2018-08-06,0.082278,0.083262,0.078055,0.079166,1,0,0.083262
243,2018-08-07,0.078118,0.080420,0.077722,0.078182,1,0,0.080420
244,2018-08-08,0.080055,0.081420,0.076626,0.077277,1,0,0.081420
245,2018-08-09,0.078801,0.081801,0.077134,0.081198,1,0,0.081801
246,2018-08-10,0.089834,0.096374,0.087405,0.090739,1,0,0.096374
247,2018-08-13,0.096501,0.104835,0.091120,0.101375,1,0,0.104835
248,2018-08-14,0.095977,0.099390,0.089580,0.091580,1,0,0.099390
249,2018-08-15,0.104803,0.118614,0.103010,0.105327,1,0,0.118614
250,2018-08-16,0.095707,0.095707,0.089675,0.093580,1,0,0.095707
251,2018-08-17,0.094517,0.096422,0.084611,0.086659,1,0,0.096422


In [6]:
IV_min = df['low'].min()
IV_min

0.018001690000000001

In [7]:
IV_max = df['high'].max()
IV_max

0.33704754999999997

In [8]:
IV_value = 0.1

In [9]:
IV_rank = (IV_value - IV_min) / (IV_max - IV_min) * 100
IV_rank

25.701104537134572

In [10]:
days_bellow_current_IV = df[df['low'] < IV_value].shape[0]

In [11]:
IV_percentile = days_bellow_current_IV / 252
IV_percentile

0.6468253968253969

With Optopus methods

In [12]:
symbol = 'SPX'
u = IndexAsset(symbol, DataSource.IB)

In [13]:
pdo(opt.historical([u], ['bar_time' ,'bar_low', 'bar_high', 'bar_open', 'bar_close', 'bar_average', 'bar_volumen', 'bar_count'])).tail()

,bar_time,bar_low,bar_high,bar_open,bar_close,bar_average,bar_volumen,bar_count
code,,,,,,,,
SPX,2018-08-13,2819.88,2843.40,2835.46,2821.93,0.0,NaN,18794
SPX,2018-08-14,2826.58,2843.11,2827.88,2839.96,0.0,NaN,18050
SPX,2018-08-15,2802.49,2827.95,2827.95,2818.37,0.0,NaN,19757
SPX,2018-08-16,2831.44,2850.49,2831.44,2840.69,0.0,NaN,18214
SPX,2018-08-17,2833.73,2855.63,2838.32,2850.13,0.0,NaN,18590


In [14]:
pdo(opt.historical_IV([u], ['bar_time', 'bar_low', 'bar_high', 'bar_open', 'bar_close', 'bar_average', 'bar_volumen', 'bar_count'])).tail()

,bar_time,bar_low,bar_high,bar_open,bar_close,bar_average,bar_volumen,bar_count
code,,,,,,,,
SPX,2018-08-13,0.091120,0.104835,0.096501,0.101375,0.104835,NaN,0
SPX,2018-08-14,0.089580,0.099390,0.095977,0.091580,0.099390,NaN,0
SPX,2018-08-15,0.103010,0.118614,0.104803,0.105327,0.118614,NaN,0
SPX,2018-08-16,0.089675,0.095707,0.095707,0.093580,0.095707,NaN,0
SPX,2018-08-17,0.084611,0.096422,0.094517,0.086659,0.096422,NaN,0


In [16]:
print(opt.IV_rank(u, 0.1))
print(opt.IV_percentile(u, 0.1))

25.701104537134572
64.68253968253968
